In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [55]:
pd.set_option('display.float', '{:.3f}'.format)
pd.options.display.max_columns = None

In [3]:
data_path = f"./data/loan_data_2007_2014.csv"

In [4]:
main_data = pd.read_csv(data_path)
data = main_data.copy() 

/tmp/ipykernel_8150/823853252.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  main_data = pd.read_csv(data_path)


In [5]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,466285.000,233142.000,134605.029,0.000,116571.000,233142.000,349713.000,466284.000
id,466285.000,13079729.906,10893711.625,54734.000,3639987.000,10107897.000,20731209.000,38098114.000
member_id,466285.000,14597660.623,11682367.441,70473.000,4379705.000,11941075.000,23001541.000,40860827.000
loan_amnt,466285.000,14317.278,8286.509,500.000,8000.000,12000.000,20000.000,35000.000
funded_amnt,466285.000,14291.801,8274.371,500.000,8000.000,12000.000,20000.000,35000.000
funded_amnt_inv,466285.000,14222.330,8297.638,0.000,8000.000,12000.000,19950.000,35000.000
int_rate,466285.000,13.829,4.358,5.420,10.990,13.660,16.490,26.060
installment,466285.000,432.061,243.486,15.670,256.690,379.890,566.580,1409.990
annual_inc,466281.000,73277.381,54963.569,1896.000,45000.000,63000.000,88960.000,7500000.000
dti,466285.000,17.219,7.851,0.000,11.360,16.870,22.780,39.990


In [6]:
data.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.000,36 months,10.650,162.870,B,B2,NaN,10+ years,RENT,24000.000,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.650,0.000,Jan-85,1.000,NaN,NaN,3.000,0.000,13648,83.700,9.000,f,0.000,0.000,5861.071,5831.780,5000.000,861.070,0.000,0.000,0.000,Jan-15,171.620,NaN,Jan-16,0.000,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.000,60 months,15.270,59.830,C,C4,Ryder,< 1 year,RENT,30000.000,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.000,0.000,Apr-99,5.000,NaN,NaN,3.000,0.000,1687,9.400,4.000,f,0.000,0.000,1008.710,1008.710,456.460,435.170,0.000,117.080,1.110,Apr-13,119.660,NaN,Sep-13,0.000,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.000,36 months,15.960,84.330,C,C5,NaN,10+ years,RENT,12252.000,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.720,0.000,Nov-01,2.000,NaN,NaN,2.000,0.000,2956,98.500,10.000,f,0.000,0.000,3003.654,3003.650,2400.000,603.650,0.000,0.000,0.000,Jun-14,649.910,NaN,Jan-16,0.000,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.000,36 months,13.490,339.310,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.000,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.000,0.000,Feb-96,1.000,35.000,NaN,10.000,0.000,5598,21.000,37.000,f,0.000,0.000,12226.302,12226.300,10000.000,2209.330,16.970,0.000,0.000,Jan-15,357.480,NaN,Jan-15,0.000,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.000,60 months,12.690,67.790,B,B5,University Medical Group,1 year,RENT,80000.000,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.940,0.000,Jan-96,0.000,38.000,NaN,15.000,0.000,27783,53.900,38.000,f,766.900,766.900,3242.170,3242.170,2233.100,1009.070,0.000,0.000,0.000,Jan-16,67.790,Feb-16,Jan-16,0.000,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   id                           466285 non-null  int64  
 2   member_id                    466285 non-null  int64  
 3   loan_amnt                    466285 non-null  int64  
 4   funded_amnt                  466285 non-null  int64  
 5   funded_amnt_inv              466285 non-null  float64
 6   term                         466285 non-null  object 
 7   int_rate                     466285 non-null  float64
 8   installment                  466285 non-null  float64
 9   grade                        466285 non-null  object 
 10  sub_grade                    466285 non-null  object 
 11  emp_title                    438697 non-null  object 
 12  emp_length                   445277 non-null  object 
 13 

In [8]:
data.dtypes.value_counts()

float64    46
object     22
int64       7
Name: count, dtype: int64

In [44]:
data.select_dtypes(include="object").columns.values

array(['grade', 'sub_grade', 'emp_title', 'home_ownership',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state',
       'earliest_cr_line', 'initial_list_status', 'last_pymnt_d',
       'next_pymnt_d', 'last_credit_pull_d', 'application_type'],
      dtype=object)

First lets confirm that our contunous varibles are numeric. Lets get rid of stirngs and alphanumeircs. Strip the number

In [9]:
data.grade.unique()

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

In [10]:
data.term.unique()

array([' 36 months', ' 60 months'], dtype=object)

In [11]:
def transform_term(val): return np.float32(re.search(r'\d+', str(val)).group()) if re.search(r'\d+', str(val)) else np.float32(0)

In [12]:
data["term"] = data["term"].apply(transform_term)

In [13]:
print(type(data.term[0]))
data.term.unique()

<class 'numpy.float32'>


array([36., 60.], dtype=float32)

In [14]:
data.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [15]:
def transform_empl_length(val):
    return np.float64(re.search(r'\d+', str(val)).group()) if re.search(r'\d+', str(val)) else np.float64(0)

In [16]:
data["emp_length"] = data["emp_length"].apply(transform_empl_length)

In [17]:
print(type(data["emp_length"][0]))
data.emp_length.unique()

<class 'numpy.float64'>


array([10.,  1.,  3.,  8.,  9.,  4.,  5.,  6.,  2.,  7.,  0.])

earliest_cr_line: The month the borrower's earliest reported credit line was opened. Here we will count the number of months past since the earliest credit line was issued.

In [18]:
data.earliest_cr_line

0         Jan-85
1         Apr-99
2         Nov-01
3         Feb-96
4         Jan-96
           ...  
466280    Apr-03
466281    Jun-97
466282    Dec-01
466283    Feb-03
466284    Feb-00
Name: earliest_cr_line, Length: 466285, dtype: object

In [19]:
data.earliest_cr_line.isna().sum()

29

In [30]:
def transform_date_by_month(val):
    if pd.isna(val):
        return pd.NA  # Return NaN if the input is NaN
    else:
        converted = pd.to_datetime(val, format="%b-%y")
        months_difference = (pd.to_datetime("2023-11-01") - converted) / pd.Timedelta(4, 'W')
        rounded_months = round(pd.to_numeric(months_difference))
        return rounded_months


In [21]:
data["earliest_cr_line_date"] = pd.to_datetime(data["earliest_cr_line"], format="%b-%y")

In [22]:

data["months_since_earliest_cr_line"] = round(pd.to_numeric((pd.to_datetime("2023-11-01") - data["earliest_cr_line_date"])/ pd.Timedelta(4, 'W')))
data["months_since_earliest_cr_line"].describe()

count   466256.000
mean       337.510
std        102.157
min       -588.000
25%        276.000
50%        322.000
75%        387.000
max        715.000
Name: months_since_earliest_cr_line, dtype: float64

We have detected the negative time since the earliest credit line date. Lets explore more.

In [23]:
data["earliest_cr_line_date"].isna().sum()

29

In [24]:
data.loc[:, ["earliest_cr_line", "earliest_cr_line_date", "months_since_earliest_cr_line"]][data["months_since_earliest_cr_line"] < 0]

,earliest_cr_line,earliest_cr_line_date,months_since_earliest_cr_line
1580,Sep-62,2062-09-01,-507.000
1770,Sep-68,2068-09-01,-585.000
2799,Sep-64,2064-09-01,-533.000
3282,Sep-67,2067-09-01,-572.000
3359,Feb-65,2065-02-01,-538.000
...,...,...,...
464003,Jan-68,2068-01-01,-576.000
464260,Jul-66,2066-07-01,-557.000
465100,Oct-67,2067-10-01,-573.000
465500,Sep-67,2067-09-01,-572.000


We can see that many rows have future time time such as Sep-62. These variables are incorreectly converted into Sept 2062 - becuase the origin of the built-in time scale starts after 1970. We colud solve this issue by preprocessing the data or impute with maximum values of this variable. I will choose later.

In [25]:
data.loc[data["months_since_earliest_cr_line"] < 0, "months_since_earliest_cr_line"] = data["months_since_earliest_cr_line"].max()
data["months_since_earliest_cr_line"].min()

157.0

Lets look at issued at date (issue_d).

In [36]:
data.issue_d

0         Dec-11
1         Dec-11
2         Dec-11
3         Dec-11
4         Dec-11
           ...  
466280    Jan-14
466281    Jan-14
466282    Jan-14
466283    Jan-14
466284    Jan-14
Name: issue_d, Length: 466285, dtype: object

In [40]:
data["issue_d_date"] = pd.to_datetime(data["issue_d"], format="%b-%y")
data["months_since_issied_date"] = round(pd.to_numeric((pd.to_datetime("2023-11-01") - data["issue_d_date"])/ pd.Timedelta(4, "W")))
print(type(data["months_since_issied_date"][0]))
data["months_since_issied_date"].describe()

<class 'numpy.float64'>


count   466285.000
mean       132.860
std         15.627
min        116.000
25%        122.000
50%        128.000
75%        139.000
max        214.000
Name: months_since_issied_date, dtype: float64

In [49]:
objLs = data.select_dtypes(include="object").columns.values
for col in objLs:
    print(f"{data[col].value_counts()}\n==========================================================")

grade
B    136929
C    125293
D     76888
A     74867
E     35757
F     13229
G      3322
Name: count, dtype: int64
sub_grade
B3    31686
B4    30505
C1    26953
C2    26740
B2    26610
C3    25317
B5    25252
C4    24105
B1    22876
C5    22178
A5    21757
D1    19261
A4    19045
D2    17046
D3    14916
D4    14099
A3    12568
D5    11566
A2    10956
A1    10541
E1     9033
E2     8669
E3     6976
E4     5992
E5     5087
F1     3940
F2     3001
F3     2708
F4     2067
F5     1513
G1     1109
G2      823
G3      583
G4      422
G5      385
Name: count, dtype: int64
emp_title
Teacher                              5399
Manager                              4438
Registered Nurse                     2316
RN                                   2204
Supervisor                           1967
                                     ... 
The Eastern Arizona AHEC                1
Delaware Quarries.com                   1
McCauley trucking and warehousing       1
save mart supermarkets                  

Lets explore discrete variables. I am going to use one-hot encoding for this exploration. The feature variable will increase by k-1 for each category in each variable. Variables such as:
- grade
- sub_grade
- home_ownership
- verification status 
- loan_status 
- purpose
- addr_state
- initial_list_status

In [50]:
one_hot_data = []
one_hot_variables = ["grade", "sub_grade", "home_ownership", "verification_status", "loan_status", "purpose", "addr_state", "initial_list_status"]

for category in one_hot_variables:
    one_hot_data.append(pd.get_dummies(data[category], prefix=category, prefix_sep="_"))

one_hot_data = pd.concat(one_hot_data, axis=1)
print(f"shape: {one_hot_data.shape}, type: {type(one_hot_data)}")

shape: (466285, 126), type: <class 'pandas.core.frame.DataFrame'>


concatenate with dataframe.

In [52]:
data =pd.concat([data, one_hot_data], axis=1)
print(f"new shape:{data.shape}")
data.columns.values

new shape:(466285, 205)


array(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'veri

Now, lets deal with missing values.

In [59]:
pd.options.display.max_rows = None
data.isnull().sum().sort_values(ascending=False)

open_il_6m                                                         466285
max_bal_bc                                                         466285
annual_inc_joint                                                   466285
dti_joint                                                          466285
verification_status_joint                                          466285
open_acc_6m                                                        466285
open_il_24m                                                        466285
mths_since_rcnt_il                                                 466285
total_bal_il                                                       466285
il_util                                                            466285
open_rv_12m                                                        466285
open_rv_24m                                                        466285
open_il_12m                                                        466285
all_util                              

lets find out the missing number and drop the varibale id it is missing more than 70% of data.

In [71]:
droppables = []
imputabels = []

for variable in data.columns.values:
    ratio = data[variable].isnull().sum() / data.shape[0]
    if ratio > 0.7: droppables.append(variable)
    if ratio > 0 and ratio < 0.7: imputabels.append(variable)

print(f"drops ({len(droppables)}): {droppables}\nimputables ({len(imputabels)}): {imputabels}")

drops (20): ['desc', 'mths_since_last_record', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m']
imputables (21): ['emp_title', 'annual_inc', 'title', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_util', 'total_acc', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'earliest_cr_line_date', 'months_since_earliest_cr_line']


In [72]:
print(f"Before dropping cols: {data.shape}")
data.drop(columns=droppables, inplace=True)
print(f"After dropping cols: {data.shape}")

Before dropping cols: (466285, 205)
After dropping cols: (466285, 185)


Total revolving high credit limit / credit limit, Failry, the missing values are equal to funded_amnt. Similarly, we have only 4 missing entries in annual_inc variable. It would be reasonable to fill with mean values,

In [73]:
data['total_rev_hi_lim'].fillna(data['funded_amnt'], inplace=True)

In [74]:
data["annual_inc"].describe()

count    466281.000
mean      73277.381
std       54963.569
min        1896.000
25%       45000.000
50%       63000.000
75%       88960.000
max     7500000.000
Name: annual_inc, dtype: float64

In [75]:
data["annual_inc"].fillna(data["annual_inc"].mean(), inplace=True)

In [78]:
# picked numeric faetures 
cols_to_impute_with_zeros = ["months_since_earliest_cr_line", "acc_now_delinq", "total_acc", "pub_rec", "inq_last_6mths", "delinq_2yrs"]
for col in cols_to_impute_with_zeros:
    data[col].fillna(0, inplace=True)

In [81]:
data.isnull().sum().sort_values(ascending=False)

mths_since_last_delinq                                             250351
next_pymnt_d                                                       227214
tot_cur_bal                                                         70276
tot_coll_amt                                                        70276
emp_title                                                           27588
last_pymnt_d                                                          376
revol_util                                                            340
collections_12_mths_ex_med                                            145
last_credit_pull_d                                                     42
open_acc                                                               29
earliest_cr_line                                                       29
earliest_cr_line_date                                                  29
title                                                                  21
loan_status_Charged Off               